# 1.Mini Batch and Batch Size
- 에포크(Epoch)는 전체 훈련 데이터가 학습에 한 번 사용된 주기를 말한다고 언급한 바 있습니다.
- 미니 배치의 크기를 배치 크기(batch size)라고 합니다.
  

- 전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 '배치 경사 하강법'이라고 부릅니다.   
반면, 미니 배치 단위로 경사 하강법을 수행하는 방법을 '미니 배치 경사 하강법'이라고 부릅니다.
------------------------------------
- 배치 경사 하강법은 경사 하강법을 할 때,  
 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 계산량이 너무 많이 듭니다.  
- 미니 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터의 일부만을 보고 수행하므로 최적값으로 수렴하는 과정에서 값이 조금 헤매기도 하지만 훈련 속도가 빠릅니다.
- 배치 크기는 보통 2의 제곱수를 사용합니다. ex) 2, 4, 8, 16, 32, 64...   
그 이유는 CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터 송수신의 효율을 높일 수 있다고 합니다.



# 2.Iteration

![스크린샷 2020-03-20 오후 3 48 17](https://user-images.githubusercontent.com/44131043/77142039-49a57000-6ac2-11ea-9367-1e2d81fa283b.png)

- 이터레이션은 한 번의 에포크 내에서 이루어지는 매개변수인 가중치 W와 b의 업데이트 횟수입니다.
- 전체 데이터가 2,000일 때 배치 크기를 200으로 한다면 이터레이션의 수는 총 10개
- 한 번의 에포크 당 매개변수 업데이트가 10번

# 3. Data Load

- 파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공
- 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다
-----------
- 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.
----------
- Dataset을 커스텀하여 만들 수도 있지만 여기서는 __텐서를 입력받아 Dataset의 형태로 변환__해주는 TensorDataset을 사용해보겠습니다.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
#TensorDataset은 기본적으로 텐서를 입력으로 받습니다. 텐서 형태로 데이터를 정의합니다.
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

In [28]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

print(x_train.shape, y_train.shape)

torch.Size([5, 3]) torch.Size([5, 1])


In [0]:
dataset = TensorDataset(x_train, y_train)

파이토치의 데이터셋을 만들었다면 데이터로더를 사용 가능합니다.  
__데이터로더는 기본적으로 2개의 인자를 입력받는다. 하나는 데이터셋, 미니 배치의 크기입니다__.  
이때 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다. (ex) 64, 128, 256...) 그리고 추가적으로 많이 사용되는 인자로 shuffle이 있습니다. __shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿉__니다.


사람도 같은 문제지를 계속 풀면 어느 순간 문제의 순서에 익숙해질 수 있습니다. 예를 들어 어떤 문제지의 12번 문제를 풀면서, '13번 문제가 뭔지는 기억은 안 나지만 어제 풀었던 기억으로 정답은 5번이었던 것 같은데' 하면서 문제 자체보단 순서에 익숙해질 수 있다는 것입니다. __그럴 때 문제지를 풀 때마다 문제 순서를 랜덤으로 바꾸면 도움__이 될 겁니다. 마찬가지로 __모델이 데이터셋의 순서에 익숙해지는 것을 방지__하여 학습할 때는 이 옵션을 True를 주는 것을 권장합니다.

In [0]:
#3개 
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
#batch_size =2 -> 3개로 나눠짐(0,1,2)

In [0]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

```python
for i in dataloader:
    print(i)
#######################
[tensor([[73., 66., 70.],
        [73., 80., 75.]]), tensor([[142.],
        [152.]])]
[tensor([[ 96.,  98., 100.],
        [ 93.,  88.,  93.]]), tensor([[196.],
        [185.]])]
```


In [0]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):

    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx)
        # print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))

In [39]:
new_var = torch.FloatTensor([73, 80, 75])
pred_y = model(new_var)
print(pred_y)

tensor([153.5911], grad_fn=<AddBackward0>)
